# 使用 SageMaker Serverless 部署向量模型

## 上传空自定义推理代码

In [ ]:

!rm -f vector_model.tar.gz
!tar zcvf vector_model.tar.gz -T /dev/null
!aws s3 cp vector_model.tar.gz s3://cloudbeer-aigc-works/search_bot/vector_model.tar.gz

## 部署模型

### 声明变量等

下面的部署使用了 SageMaker Serverless 部署模型

In [3]:
import boto3  
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.serverless import ServerlessInferenceConfig

s3_model = "s3://cloudbeer-aigc-works/search_bot/vector_model.tar.gz"

iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='HuggingfaceExecuteRole')['Role']['Arn']
endpoint_name='text2vector'

### 部署

In [4]:
huggingface_model = HuggingFaceModel(
  model_data=s3_model,
  role=role,
	transformers_version='4.26',
	pytorch_version='1.13',
	py_version='py39',
  entry_point='inference.py',
  source_dir='src/vector',
)

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=4096, max_concurrency=10,
)

predictor = huggingface_model.deploy(
  serverless_inference_config=serverless_config,
  endpoint_name=endpoint_name
)

----!

# 测试预测

In [5]:
from sagemaker.huggingface.model import HuggingFacePredictor

from numpy import dot
from numpy.linalg import norm

def numpy_sim(v1, v2):
    return dot(v1, v2)/(norm(v1)*norm(v2))

predictor = HuggingFacePredictor(
  endpoint_name=endpoint_name
)

In [6]:
res = predictor.predict({
    "text": "Hello world."
})

print(len(res), res[:10])

768 [0.007056477479636669, 0.12021193653345108, -0.010267889127135277, 0.08088284730911255, 0.21164970099925995, 0.019357632845640182, -0.041434358805418015, 0.03360966965556145, 0.0508563332259655, 0.003544007195159793]


相似性测试

In [8]:
word = "对不起"
compares = ["抱歉", "吃了吗", "I'm sorry", "the weather",  "ごめんなさい", "미안해", "Désolé"]

wordvec = predictor.predict({
    "text": word
})


for compare in compares:
  print(f"{word} 与 {compare} 的相似度：",  numpy_sim(predictor.predict({
      "text": compare
  }), wordvec))

对不起 与 抱歉 的相似度： 0.9974859157398233
对不起 与 吃了吗 的相似度： 0.4769853065958965
对不起 与 I'm sorry 的相似度： 0.9374510242687436
对不起 与 the weather 的相似度： 0.18689081182255046
对不起 与 ごめんなさい 的相似度： 0.996968684947639
对不起 与 미안해 的相似度： 0.9881213472963395
对不起 与 Désolé 的相似度： 0.9622101617182905


删除模型

In [ ]:

from sagemaker import  Predictor
predictor = Predictor(
    endpoint_name=endpoint_name
)

predictor.delete_model()
predictor.delete_endpoint()